Film Agent

#### > Setup

In [ ]:
!pip freeze | grep boto3

### Analyze and extract video

In [ ]:
import boto3
import json
import uuid
import sagemaker
import uuid

boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)
if sess.default_bucket():
    bucket = sess.default_bucket()
else:
    bucket = "<YOUR-BUCKET-NAME>" # Provide your own bucket
region = sess.boto_region_name
prefix = "media-operations-agent-claude"

bda_client = boto_session.client('bedrock-data-automation')
bda_runtime_client = boto_session.client('bedrock-data-automation-runtime')
s3_client = boto_session.client('s3')

#access account id
sts_client = boto_session.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

default_profile_arn = f"arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1"

### Prepare the sample video

In [ ]:
video_file = 'NetflixMeridian.mp4'
!curl "https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/Netflix_Open_Content_Meridian.mp4" --output NetflixMeridian.mp4

In [ ]:
s3_key = f"{prefix}/{video_file}"

s3_client.upload_file(video_file, bucket, s3_key)
film_video_s3_path = f"s3://{bucket}/{s3_key}"

### Create a BDA project

In [ ]:
response = bda_client.create_data_automation_project(
    projectName=f'{prefix}-{str(uuid.uuid4())[0:4]}',
    projectDescription='Media operations agent',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['TEXT_DETECTION','TRANSCRIPT'],
                },
                'boundingBox': {
                    'state': 'DISABLED',
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': ['VIDEO_SUMMARY','CHAPTER_SUMMARY','IAB'],
            }
        }
    }
)

video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

### Extract analysis from video

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{bucket}/{prefix}/outputs'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=default_profile_arn
)

invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

### Wait

In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

### Access the BDA analaysis result

In [ ]:
def read_json_on_s3(s3_uri, s3_client):
    # Parse s3 bucket and key from s3 uri
    s3_bucket = s3_uri.split('/')[2]
    s3_key = s3_uri.replace(f's3://{s3_bucket}/','')
    
    # Read BDA output_config file on S3
    response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
    file_content = response['Body'].read().decode('utf-8')  # Read the content and decode it to a string
    # Convert the content to JSON
    return json.loads(file_content)

In [ ]:
config_data = read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

In [ ]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = read_json_on_s3(result_uri,s3_client)

JSON(result_data)

In [ ]:
video_analysis = {
    "visual_summary": result_data["video"]["summary"],
    "audio_transcription":result_data["video"]["transcript"]["representation"]["text"],
    "metadata":result_data["metadata"]
    
}
video_analysis

## Setup Film Agent

In [ ]:
import os
import time
import sys

account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
]

In [ ]:
agent_name = f"f-agent-{agent_suffix}"
lambda_name = f"fn-f-agent-{agent_suffix}"

agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

knowledge_base_name = f'{agent_name}-02'

suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information of all the films"
bucket_name = f'{agent_name}-{suffix}'

cast_table = f'cast-table-{agent_suffix}'
cast_pk = 'id'
cast_sk = 'name'
env_args = [cast_table, cast_pk, cast_sk]
bucket_name

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
!pip install retrying

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Upload Film Documents to S3

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
films_folder = "films"

!aws s3 sync {films_folder} s3://{bucket_name}/

In [ ]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when you need to look up title, director, and plot of a film"""
}

## Creating Agent

In [ ]:
agent_description = """You are a film analyst. You job is analyzing shorts or derivatives of 
films, like promo or trailer. Finding the matching film and identify key cast memebers."""

agent_instruction = """
Your task is to analyze film videos or deriative of film videos and match it to the correct
film title and identify key cast memebers.

Your capabilities include:
<capabilities>
- Matching video to correct film title
- Identify key cast members
</capabilities>

Response style:
<Response style>
- Be helpful and solution-oriented
- Use clear, non-technical language
- Maintain natural conversation flow
- Be concise yet informative
- do not add extra information not required by the user
</Response style>
"""

film_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(20)
film_agent

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    film_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)
time.sleep(20)

## Test Knowledge Base W/ Agent

In [ ]:
session_state = {
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

In [ ]:
%%time
import uuid

response = agents.invoke(
    input_text=f"which film is directored by Curtis Clark", 
    agent_id=film_agent[0], 
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

In [ ]:
%store knowledge_base_name
%store kb_config
%store agent_instruction
%store agent_name
%store video_analysis

### Creating Lambda

In [ ]:
%%writefile detection.py
import boto3
import json
import os
import time
from boto3.dynamodb.conditions import Key, Attr

## DynamoDB parameters
dynamodb_resource = boto3.resource('dynamodb')
cast_table = os.getenv('cast_table')
cast_pk = os.getenv('cast_pk')

## Rekognition parameters
rek_client = boto3.client('rekognition')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except StopIteration:
        raise ValueError(f"Required parameter '{name}' not found in event")
        
def get_cast_member(cast_id):
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {cast_table}.')

def start_celebrity_detection(bucket, video_key):
    response = rek_client.start_celebrity_recognition(
        Video={
            'S3Object': {
                'Bucket': bucket,
                'Name': video_key
            }
        }
    )
    return response['JobId']

def get_celebrity_detection_results(job_id):
    response = rek_client.get_celebrity_recognition(JobId=job_id)
    return response

def extract_bucket_key(video_s3_path):
    path = video_s3_path[5:]  # Remove 's3://'
    bucket, key = path.split('/', 1)  # Split into bucket and key   
    return bucket, key

def detect_key_figures(video_s3_path):
    bucket, key = extract_bucket_key(video_s3_path)
    
    job_id = start_celebrity_detection(bucket, key)
    print(f"Started celebrity detection job: {job_id}")

    while True:
        response = get_celebrity_detection_results(job_id)
        status = response['JobStatus']
        
        if status in ['SUCCEEDED', 'FAILED']:
            print("JOB COMPLETE....")
            break
        
        print("Job in progress...")
        time.sleep(10)

    unique_celebrities = {}  # Dictionary to store unique celebrities

    if status == 'SUCCEEDED':
        for celebrity in response['Celebrities']:
            celeb = celebrity['Celebrity']
            
            # Only process celebrities with 95%+ confidence
            if celeb['Confidence'] >= 95.0:
                celeb_id = celeb['Id']
                
                # Store or update celebrity info only if not already stored
                if celeb_id not in unique_celebrities:
                    celebrity_info = {
                        'name': celeb['Name'],
                        'confidence': celeb['Confidence'],
                        'id': celeb_id,
                        'first_appearance': celebrity['Timestamp']
                    }
                    
                    # If you have additional celebrity info in DynamoDB
                    try:
                        query_items = get_cast_member(celeb_id)
                        if query_items:
                            celebrity_info.update(query_items[0])
                    except Exception as e:
                        print(f"Error fetching additional celebrity info: {str(e)}")
                    
                    unique_celebrities[celeb_id] = celebrity_info
    else:
        print("Detection failed....")

    # Convert the dictionary values to a list for the final output
    final_output = list(unique_celebrities.values())
    return final_output

def populate_function_response(event, response_body):
    return {
        'response': {
            'actionGroup': event['actionGroup'],
            'function': event['function'],
            'functionResponse': {
                'responseBody': {
                    'TEXT': {
                        'body': str(response_body)
                    }
                }
            }
        }
    }
    
def lambda_handler(event, context):
    print(event)
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    video_s3_path = get_named_parameter(event, "video_s3_path")

    if function == 'detect_key_figures':
        result = detect_key_figures(video_s3_path)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

In [ ]:
functions_def = [
    {
        "name": "detect_key_figures",
        "description": """Detect key figures (celebrity, cast member) from a video
        and retrieve information about their position and team""",
        "parameters": {
            "video_s3_path": {
                "description": "S3 location of the video (e.g: s3://......)",
                "required": True,
                "type": "string"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_lambda_action_group_with_rek(
    agent_name=agent_name,
    lambda_function_name=lambda_name,
    source_code_file="detection.py",
    agent_functions=functions_def,
    agent_action_group_name="key_figure_detection_actions",
    agent_action_group_description="Functions to identify key figures and look up cast members and their role from the video",
    env_args=env_args
)
time.sleep(30)

### > Loading dynamoDB

In [ ]:
# Open and read the JSONL file
with open('cast_members.jsonl', 'r') as file:
    table_items = [json.loads(line.strip()) for line in file]

agents.load_dynamodb(cast_table, table_items)

In [ ]:
resp = agents.query_dynamodb(
    cast_table, cast_pk, '4kn3Xu8r'
)
resp

In [ ]:
%store cast_table
%store cast_pk
%store film_video_s3_path

### Test Agent

In [ ]:
session_state = {
    'promptSessionAttributes': {
        "<video_analysis>": json.dumps(video_analysis),
        "<video_s3_path>": film_video_s3_path,
    },
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

In [ ]:
%%time
response = agents.invoke(
    input_text=f"""
    Here is a clip extraction:
    $prompt_session_attributes.video_analysis$

    if needed, here is the s3 location of the video:
    $prompt_session_attributes.video_s3_path$
    
    can you tell me which film is this clip from?
    """, 
    agent_id=film_agent[0],
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

### Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
film_agent_alias_id, film_agent_alias_arn = agents.create_agent_alias(
    film_agent[0], 'v1'
)

### Saving Information

In [ ]:
film_agent_arn = agents.get_agent_arn_by_name(agent_name)
film_agent_id = film_agent[0]
film_kb = knowledge_base_name
film_agent_name = agent_name
film_kb_id = kb_config['kb_id']
film_kb_arn = kb_arn
film_kb_description = knowledge_base_description
film_kb_name = knowledge_base_name
film_video_analysis = video_analysis

%store film_agent_alias_id
%store film_agent_alias_arn
%store film_agent_arn
%store film_agent_id
%store film_kb
%store film_agent_name
%store film_kb_id
%store film_kb_arn
%store film_kb_description
%store film_kb_name

%store film_video_analysis
%store film_video_s3_path

### Clean Up

In [ ]:
# agents.delete_agent(agent_name)

In [ ]:
# kb.delete_kb(knowledge_base_name)

In [ ]:
# agents.delete_lambda(lambda_function_name=lambda_name)

In [ ]:
# #delete BDA project
# response = bda_client.delete_data_automation_project(
#     projectArn=video_project_arn
# )